In [ ]:
## Install all required packages
!pip install nltk
!pip install umap-learn
!pip install bertopic
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 33.7 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=8ecff2c98e2666fd463eb2192bc4e7ee2c9d9e72374a42798100f3062225b92f
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=8e9071abcc5a2b1fb0f5620c6cfa058b3dc1033b89a6674692c4af747b53d8a0
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8b4be1ad6d21bf5019235c2eff65928
Successfully built umap-learn pynndescent
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████

In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [5]:
## import required libraries
from google.colab import drive
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import enchant

from umap import UMAP
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
## mount the working directory
drive.mount("text_clustering")

Drive already mounted at text_clustering; to attempt to forcibly remount, call drive.mount("text_clustering", force_remount=True).


In [ ]:

## read the dataset
df = pd.read_excel("/content/text_clustering/MyDrive/text_clustering/data_with_keywords.xlsx")


In [ ]:
df.head(5)

,Unnamed: 0,date,username,content,key
0,0,2022-09-29 23:59:55+00:00,pdonahue1107,@JamesGoldberg40 @PartNoWays @Quinny1S @KAGMAM...,accident
1,1,2022-09-29 23:59:53+00:00,GUTVIRUS,accident,accident
2,2,2022-09-29 23:59:44+00:00,xeph_s,i got hospitalized because i touched one by ac...,accident
3,3,2022-09-29 23:59:39+00:00,catsmokesesh,@kaylaabaela I did it on accident bruh,accident
4,4,2022-09-29 23:59:38+00:00,callmeelliss,not me involving Future by accident💀💀,accident


In [ ]:
# filter required columns
df_1 = df[['content', 'key']]
df_2 = df_1.sample(frac=0.99,random_state=0) # To sheffle the dataset

In [ ]:
## pre-define function for cleaning the tweets
d = enchant.Dict("en_US")
lemmetizer = WordNetLemmatizer()
def clean_tweet(text):
    clean_text = text.lower()
    clean_text = re.sub("'", "", clean_text) # to avoid removing contractions in english
    clean_text = re.sub("@[A-Za-z0-9_]+","", clean_text)
    clean_text = re.sub("#[A-Za-z0-9_]+","", clean_text)
    clean_text = re.sub(r'http\S+', '', clean_text)
    clean_text = re.sub('[()!?]', ' ', clean_text)
    clean_text = re.sub('\[.*?\]',' ', clean_text)
    clean_text = re.sub("[^a-zA-Z]"," ", clean_text)
    clean_text = clean_text.strip().split()
    clean_text = [i for i in clean_text if d.check(i)==True and len(i)>2 ]
    clean_text_2 = [lemmetizer.lemmatize(word) for word in clean_text if word not in set(stopwords.words("english"))]
    if len(clean_text_2)>2:
      clean_text_3 = " ".join(clean_text_2)
    else:
       clean_text_3 = np.nan

    return clean_text_3

In [ ]:
## apply the cleaning function to the content column
df_2["content_clean"] = df_2["content"].apply(lambda x:clean_tweet(x))

In [ ]:
## drop duplicates and null raws
df_2_1 = df_2.drop_duplicates(subset=["content_clean"])
df_2_2= df_2_1[df_2_1["content_clean"].notna()]

In [ ]:
# check the shapes of original and clean dataframes
df.shape, df_2_2.shape

((25577, 5), (21938, 3))

In [ ]:
df_2_2.head()

,content,key,content_clean
11564,@mmattheww1127 @James02739108 @nikolas5_17 @yc...,racism,racism natural state socialized
25201,Giving To Services is a decentralized service ...,medical emergency,giving service decentralized service harness t...
10761,@JasonPYYC what they're disavowing isn't reall...,racism,disavowing really racism getting caught
16827,@MazeAdhara @niccoyat “Somebody close to me di...,poverty,somebody close died decent person company need...
13244,Bring the Flood trays already damn,floods,bring flood tray already damn


In [ ]:
# save the clean data
# df_2_2.to_excel("/content/text_clustering/MyDrive/text_clustering/data_clean.xlsx")

**Embedding Models**

sentence transformers has selected for this usecase because its work better for most of the dataset. 
https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html

In [ ]:
# initialize the sentence transformer model using pre-defined model "all-MiniLM-L6-v2"
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
sentence_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

**Dimentionality Reduction**

UMAP is used for Dimentionality reduction. PCA is already used in clustering.
Find the optimal parameters of UMAP

###### **There are two deciding parameters**

•	n_neighbors is the local neighborhood size for UMAP. This is the parameter that controls the local versus global structure in data.

1.	A low value forces UMAP to focus more on the local structure and may lose insights into the big picture.
2.	A high value pushes UMAP to look at the broader neighborhoods and may lose details on local structure. This usually results in a larger cluster size.

•	n_components -  indicates the output dimension for UMAP. This is the dimension of data that will be passed into the clustering model.



In [ ]:
# List of data use to fit the model
data = df_2_2["content_clean"].tolist()

*  
The following set of codes use to fit UMAP models for different combinations of defined list of n_neigh and n_comp.

Once parameters of UMAP has changed, BERTopic model also changed. Therfore each model fit different topics which needed to be analysed to select the better values of above parameters.

We can check which values of n_neigh and n_comp are give better BERTopic model.

Since this step is done and found the optimal values for those parameters, no need to run again. 

If the dataset has changed or this may not the optimal values, then uncomment and run this set of codes again and find the better values of the parameters.

In [ ]:
# n_neigh =[10,20,15,30,50]
# n_comp = [3,5,10,15,20]
# freq_all =[]
# for i in tqdm(n_neigh):
#   for j in tqdm(n_comp):
#     print(i,j)
#     umap_model = UMAP(n_neighbors=i, n_components=j, min_dist=0.0, metric='cosine',random_state=0)
#     topic_model = BERTopic(embedding_model=sentence_model,umap_model=umap_model,calculate_probabilities=True)
#     topics, probabilities = topic_model.fit_transform(data)
#     freq = topic_model.get_topic_info()
#     freq["combination_n_neigh_n_comp"] = str([i,j])
#     freq_all.append(freq)
#     # bar_chart_words =topic_model.visualize_barchart(top_n_topics=20)
#     # bar_chart_words.write_html("/content/text_clustering/MyDrive/text_clustering/BERT_Vis/bar_chart_n_neighbors_{}_n_components_{}.html".format(i,j))
#     # inter_distance_map  = topic_model.visualize_topics()
#     # inter_distance_map.write_html("/content/text_clustering/MyDrive/text_clustering/BERT_Vis/inter_distance_map_n_neighbors_{}_n_components_{}.html".format(i,j))

# df_frq_all = pd.concat(freq_all)
# df_frq_all.to_excel("/content/text_clustering/MyDrive/text_clustering/freq_hyperparameter_UMAP_new_1.xlsx")


  0%|          | 0/5 [00:00<?, ?it/s]

10 3



 20%|██        | 1/5 [04:45<19:01, 285.32s/it]

10 5



 40%|████      | 2/5 [09:41<14:35, 291.98s/it]

...


 60%|██████    | 3/5 [14:48<09:56, 298.34s/it]

50 15



 80%|████████  | 4/5 [20:03<05:05, 305.21s/it]

50 20



100%|██████████| 5/5 [2:05:55<00:00, 1511.01s/it]


Once the dataframes of Topics, count and Names for each combination of n_neighbors and n_components has saved in the given location, it can be reviewed to select the correct combination of n_neighbors and n_components.

When selecting the comination of parameters following factors are considered.

*   relevance of the words per topic
*   within lower number of topics higher number of data points captured

In [ ]:
# So, UMAP model has initilized with the values of 20 and 3 for n_neighbors and n_components respectively.
umap_model_selected = UMAP(n_neighbors=20, n_components=3, min_dist=0.0, metric='cosine',random_state=0)

**Clustering Model**

Default HDBSCAN is used and try to tune its parameter for BERTopic model. 

KMeans is already used in clustering. If this model fails to give good prediction or dataset has changed then KMeans can be tested.

**Tune the following parameters..**

•	min_cluster_size -  the minimum number of documents in each cluster.

•	min_samples controls the number of outliers. It defaults to the same value as min_cluster_size. Reducing the value helps to reduce the noise in the topics.



*   Similary in UMAP, parameters of HDBSCAN has selected by running the following set of codes. 

If there is any changes in dataset or model is not performaing well in prediction of new data points, uncomment and run this set of code again and fine tune the parameters.

In [ ]:
# list_min_cluster_size =[10,15,20,30,50]
# list_min_samples = [10,15,20,30]
# freq_hdbscan=[]
# for c in tqdm(list_min_cluster_size):
#   for s in tqdm(list_min_samples):

#     hdbscan_model = HDBSCAN(min_cluster_size=c, min_samples = s, metric='euclidean', prediction_data=True)
#     topic_model = BERTopic(embedding_model=sentence_model,umap_model=umap_model_selected,hdbscan_model=hdbscan_model,
#                             calculate_probabilities=True)
#     topics, probabilities = topic_model.fit_transform(data)
#     freq = topic_model.get_topic_info()
#     freq["combination_min_cluster_size_min_sample"] = str([c,s])
#     freq_hdbscan.append(freq)

# df_hdbsac_all = pd.concat(freq_hdbscan) 

# df_hdbsac_all.to_excel("/content/text_clustering/MyDrive/text_clustering/hyperparameter_hsdbs_new_1.xlsx")

100%|██████████| 5/5 [1:19:14<00:00, 950.97s/it]


In [ ]:
# Selected parameters for hdbscan are min_custer_size=50, min_samples = 30, so
# the model is initialized with those values
c=50
s=30
hdbscan_model = HDBSCAN(min_cluster_size=c, min_samples = s, metric='euclidean', prediction_data=True,)


In [ ]:
# Now, the BERTopic model can be initialize with the above UMAP and HDBSCAN models
topic_model = BERTopic(embedding_model=sentence_model,umap_model=umap_model_selected,hdbscan_model=hdbscan_model,
                        calculate_probabilities=True)
topics, probabilities = topic_model.fit_transform(data)
freq = topic_model.get_topic_info()

In [ ]:
# check the Topic, Count and Names of the fitted BERTopic model
freq

,Topic,Count,Name
0,-1,1353,-1_medical_emergency_amp_terrorism
1,0,4125,0_poverty_people_amp_poor
2,1,3187,1_accident_car_traffic_reported
3,2,2493,2_child_abuse_kid_amber
4,3,2190,3_terrorism_terrorist_state_domestic
5,4,2075,4_racism_racist_white_black
6,5,1755,5_flood_hurricane_insurance_water
7,6,1027,6_emergency_medical_call_please
8,7,911,7_bribery_corruption_vote_former
9,8,684,8_theft_identity_document_stolen


**Merge Similar Topics**

The above dataframe shows the Topic number and it's Name. The similar Names can be found as they talked about nearly same topic, for example, Topic 6,9,10,12,14 20 are about medical emergency. So, we can merge those topics together for the final model.

In [ ]:
# select the similar topics as lists
topics_to_merge = [[6,9,10,12,14,20],[8,15],[3,11,17,19],[13,16,18,21,22,23]]
topic_model.merge_topics(data, topics_to_merge)
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,1353,-1_medical_emergency_amp_terrorism
1,0,4125,0_poverty_people_amp_poor
2,1,3187,1_accident_car_traffic_reported
3,2,2581,2_terrorism_act_state_terrorist
4,3,2493,3_child_abuse_kid_amber
5,4,2123,4_medical_emergency_abortion_woman
6,5,2075,5_racism_racist_white_black
7,6,1755,6_flood_hurricane_insurance_water
8,7,911,7_bribery_corruption_vote_money
9,8,800,8_theft_art_identity_document


In [ ]:
topic_model.visualize_barchart(top_n_topics=11)

**Top words**
##### 1. ngrams is used to specify the range of n-grams included in the topic model
##### 2. top_n_words -  controls how many words are used to describe the topic.

In [ ]:
# topic_model_ngrams = BERTopic(embedding_model=sentence_model,umap_model=umap_model_selected,hdbscan_model=hdbscan_model,
#                         calculate_probabilities=True,n_gram_range=(1,3))
# topics_ngrams, probabilities_ngrams = topic_model_ngrams.fit_transform(data)
# freq_ngrams = topic_model_ngrams.get_topic_info()

In [ ]:
# freq_ngrams 
### This may not useful to this use case
### if dataset has changed or higher prediction errors occured, uncomment the above set of codes and check the accuracy of new prdiction

**Words Universe**
###### control the number of words for the topic model. Limiting the number of words helps to reduce the noise in the topics.
###### min_df  and max_features are used for the c-TF-IDF 


*   min_df - for example, min_df=10 indicates that any words that appeared less than 10 times in the corpus will not be included in the c-TF-IDF calculation
*   max_features -  indicates the maximum number of words to include for the c-TF-IDF calculation. For example max_features=1000 means that the top 1000 words with the highest frequency in the corpus will be included.



In [ ]:
# vectorizer_model = CountVectorizer(min_df=20)

In [ ]:
# topic_model_1 = BERTopic(embedding_model=sentence_model,umap_model=umap_model_selected,hdbscan_model=hdbscan_model,
#                          vectorizer_model=vectorizer_model)
# topics_1, probabilities_1 = topic_model_1.fit_transform(data)
# freq_1 = topic_model_1.get_topic_info()

In [ ]:
# freq_1
### This may not useful to this use case
### if dataset has changed or higher prediction errors occured, uncomment the above set of codes and check the accuracy of new prdiction

##### **Prediction**

In [ ]:
# pre-defined function for clean and prediction of new_text
def prediction(text,topic_model):
  text_clean = clean_tweet(text)
  topics, probs = topic_model.transform(text_clean)
  return text_clean,topics, probs


In [ ]:
text_clean,topics, probs = prediction("Four people killed in car accident in #Sharqiya https://t.co/CS3BLn8GB2 #egypt",topic_model)

In [ ]:
text_clean,topics, probs
# This prediction is fine, new text predicted as Topic 1 since it talked about accident with the probalitiy of 0.89

('four people killed car accident',
 [1],
 array([[0.00292468, 0.8900221 , 0.0108574 , 0.00292291, 0.04321777,
         0.00260762, 0.00351477, 0.00292501, 0.00677389, 0.03423385]]))

In [ ]:
# test 2
new_text_1 = """
@PattataM @MikeNew19512115 “Condemning Zionism as racism is an affront to the Jewish people, who have been history’s 
primary victims of racism. 
It will encourage the latent anti-Semitism that has been a blot on world history.” -Cesar Chavez
"""

In [ ]:
text_clean_1,topics_1, probs_1 = prediction(new_text_1,topic_model)

In [ ]:
text_clean_1,topics_1, probs_1
# This prediction is fine, new text predicted as Topic 5 since it talked about racism

('condemning racism affront people history primary victim racism encourage latent anti blot world history',
 [5],
 array([[1.20864157e-03, 6.08204093e-04, 3.43481874e-03, 7.73765479e-04,
         3.67539412e-03, 9.83209173e-01, 6.48889957e-04, 8.25499662e-04,
         1.68059513e-03, 3.93501726e-03]]))

In [ ]:
# save the model
topic_model.save("/content/text_clustering/MyDrive/text_clustering/bertopic_model_new")

In [ ]:
# load the model
# if this model needed to be used without any training just load the model and use it.
# bertopic_model = BERTopic.load("/content/text_clustering/MyDrive/text_clustering/bertopic_model", embedding_model="all-MiniLM-L6-v2")